In [29]:
import matplotlib.pyplot as plt
import os
import numpy as np
import sys
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import shutil

def path_link(path:str):
    sys.path.append(path)

path_link('/master/code/lib')

import dataLoading as dl
from measure import plotStdMessage
import simulation_v2 as sim
import features as ft

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

yessss


In [ ]:
class Parameters():
    def __init__(self):
        self.dt = 0.001
        self.v0 = 60
        self.k = 70
        self.epsilon = 0.5
        self.tau = 3.5
        self.R = 1
        self.N = 200
        self.boundary = 100

        self.nbStep = 150


def getData(params):
    N = params.N
    v0 = params.v0
    k = params.k
    eps = params.epsilon
    tau = params.tau
    R = params.R
    dt = params.dt
    nbStep = params.nbStep
    boundary = params.boundary

    lim = 0.85 * 100

    xPos = np.linspace(-lim, lim, 10)
    yPos = np.linspace(-lim, lim, 10)
    gridX, gridY = np.meshgrid(xPos, yPos)
    delta = np.random.uniform(0, 7, gridX.shape + (2,))

    gridX2 = gridX + delta[:, :, 0]
    gridY2 = gridY + delta[:, :, 1]

    pos = np.stack([gridX.ravel(), gridY.ravel()], axis=1)
    pos_perturbed = np.stack([gridX2.ravel(), gridY2.ravel()], axis=1)

    pos = np.concatenate([pos, pos_perturbed], axis=0)

    angles = np.random.rand(pos.shape[0]) * 2 * np.pi

    data = sim.compute_main(N, (v0, tau, k, eps), boundary, T = nbStep, initialization = (pos, angles), dt = dt)[0]

    x, y, attr, inds = ft.processSimulation(data)

    dataList = []

    for i in range(len(x)):
        g = Data(x = x[i][:, 2:], y = y[i], edge_attr = attr[i], edge_index = inds[i])
        dataList.append(g)

    return dataList

In [ ]:
##############################
# paths 

#PATH = 'master/code/runs1'
#PATH = 'master/code/runs2'
#PATH = ['/master/code/analyze_models/exps/test_new_activation_0']
PATH = ['/master/code/analyze_models/exps/exp-test']

#DISPLAY_PATH = 'master/code/display_l1'
#DISPLAY_PATH = '/master/code/display_l1_2'
#DISPLAY_PATH = ['/master/code/analyze_models/display2/test_new_activation_0']
DISPLAY_PATH = ['/master/code/analyze_models/display2/exp-test']

MODEL_PATH = '/master/code/models/mod_base'


##############################




In [ ]:
def findModels(path):
    pathLists = []
    for root, dirs, files in tqdm(os.walk(path)):
            for file in files:
                  
                  if file.endswith('.pt'):
                        pathLists.append(os.path.join(root, file))


    return pathLists


def delete_wandb_dirs(start_path):
    for root, dirs, files in os.walk(start_path, topdown=False):
        for dir_name in dirs:
            if dir_name == "wandb":
                dir_path = os.path.join(root, dir_name)
                print(f"Deleting: {dir_path}")
                shutil.rmtree(dir_path)



def getName(path):
    run_name = path.split('/')[-3]

    model_type = path.split('/')[-1].split('.')[0]

    if 'best' in model_type:
        model_type = '_best'

    else:
         model_type = '_latest'

    name = run_name + model_type

    return name


def loadModel(modelName:str, inputShape:int = 8, edges_shape = 5, path = None):
    """ 
    Function to import the model

    Args:
    -----
        - `modelName`: name of the model
        - `inputShape`: inout shape of the NN
        - `edges_shape`: edge shape of the NN
        - `path`: path where the models are
    """

    sys.path.append(path)

    loadFun = __import__(f'{modelName}', fromlist = ('loadNetwork'))

    model = loadFun.loadNetwork(inputShape, edges_shape)

    return model



def getModelName(key):

    name = ''

    #if 'simplest' in key:
    #    name = name + 'simplest'
    
    name = name + 'simplest'

    ## other possibilities

    if 'no-dropout' in key:
        name = name + '_no-dropout'
    else:
        name = name + '_dropout'

    if 'no-encoder' in key:
        name = name + '_no-encoder'
    else:
        name = name + '_encoder'

    if 'relu' in key:
        name = name + '-relu'

    return name


##############################
# Messages
#TODO linear 




In [ ]:
def getOrderedVals(attribute, message, bins):
    bin_edges = np.linspace(np.min(attribute), np.max(attribute), bins + 1)
    bin_indices = np.digitize(attribute, bin_edges) - 1

    means = np.zeros(bins)
    stds = np.zeros(bins)

    for i in range(bins):
        bin_mask = bin_indices == i
        if np.any(bin_mask):
            means[i] = np.mean(message[bin_mask])
            stds[i] = np.std(message[bin_mask])
        else:
            means[i] = np.nan
            stds[i] = np.nan

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2


    return bin_centers, means, stds


def findIndices(message, nb = 5):
    stdv=plotStdMessage(message)
    plt.close()

    inds = np.argsort(stdv)
    # change of the order
    return np.flip(inds[-nb:])



def calculate_interaction(dist, rij, k, epsilon, radii = 1.0):
    """
    Given the vectors ri and rj, compute the force between them
    """


    r = dist

    # Combined radius of both particles (assume unit radii for now)
    #bij = 2.0                       # Ri + Rj 
    bij = radii + radii

    if r < bij*(1 + epsilon):
        force = k*(r - bij)*rij/r  
    elif r < bij*(1 + 2*epsilon):
        force = -k*(r - bij - 2*epsilon*bij)*rij/r
    else:
        force = torch.tensor([0.0, 0.0])
    return force

In [ ]:
def getForces(graph, k, epsilon):
    inds = graph.edge_index
    #print(inds)

    messages = []

    for i in range(inds.shape[1]):
        messages.extend(calculate_interaction(graph.edge_attr[i, 0],
                                              torch.tensor([graph.edge_attr[i, 0] * graph.edge_attr[i, 1], graph.edge_attr[i, 0] * graph.edge_attr[i, 2]]),
                                              k = k,
                                              epsilon=epsilon
                                              ).cpu().detach().numpy().tolist())
        
    return messages


def getGroundTruth(data, k, epsilon):

    res = []

    for graph in data:

        res.extend(getForces(graph, k, epsilon))


    return np.array(res)



def getPrediction(model, data, inds = None):
    res = None

    for graph in data:

        if inds is None:
            messages = model.message(graph).cpu().detach().numpy()
        else:   
            messages = model.message(graph).cpu().detach().numpy()[:, inds]
        
        
        if res is None:
            res = messages
        else:
            res = np.vstack((res, messages))

    return res


def get_messages_model(model, data, nbMax:int = 2):
    message = getPrediction(model, data, inds = None)

    inds = findIndices(message, nb = nbMax)

    return message[:, inds]





def get_sum_messages_model(model, data, inds = None):
    res = None

    for graph in data:

        if inds is None:
            messages = model.sum_message(graph).cpu().detach().numpy()
        else:   
            messages = model.sum_message(graph).cpu().detach().numpy()[:, inds]
        
        
        if res is None:
            res = messages
        else:
            res = np.vstack((res, messages))

    return res


def getInputs(data):
    res = None

    for graph in data:

        inputs = graph.x.cpu().detach().numpy()

        if res is None:
            res = inputs
        else:
            res = np.vstack((res, inputs))

    return res

def getEdges(data):
    res = None

    for graph in data:

        edges = graph.edge_attr.cpu().detach().numpy()

        if res is None:
            res = edges
        else:
            res = np.vstack((res, edges))

    return res

def getOutput(data):
    res = None

    for graph in data:

        y = graph.y.cpu().detach().numpy()

        if res is None:
            res = y
        else:
            res = np.vstack((res, y))

    return res




